In [1]:
import rectification as rec
from sklearn.datasets import load_sample_images
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import os
import pathlib



In [2]:
from utils import *
from models.model import *
import rectification as rec

Using TensorFlow backend.


In [3]:
def filterDatasetDeepSolarEye(folder):
    """[summary]

    Args:
        folder ([type]): [description]
        mode (str, optional): [description]. Defaults to 'train'.

    Returns:
        [type]: [description]
    """
    onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    unique_images = []
    for file in onlyfiles:
        file_obj = {'file_name' : "//" + file}
        unique_images.append(file_obj)
    dataset_size = len(unique_images)
    
    return unique_images, dataset_size, None

In [4]:
log = RuntimeLogger(".", 'DeepSolarEyeCalc', 0)
log.start()

In [5]:
root_folder = 'data'
mode = 'DeepSolarEyeCalc'
dataset = 'DeepSolarEye_192x192'

In [6]:
# Regression
nr_images_batch = 5000
nr_batch = 9
nr_epoch = 2

In [7]:
folder = 'data/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages'
images_deepsolareye, data_size, coco = filterDatasetDeepSolarEye(folder)


In [8]:
    input_image_size = (192,192)

    folder_out = '{}/{}/{}'.format(root_folder, mode, dataset)
    folder_out_mask = '{}/{}/{}_mask'.format(root_folder, mode, dataset)
    folder_out_rec = '{}/{}/{}_rec'.format(root_folder, mode, dataset)
    
    pathlib.Path(folder_out).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(folder_out_mask).mkdir(parents=True, exist_ok=True)
    pathlib.Path(folder_out_rec).mkdir(parents=True, exist_ok=True)

In [ ]:
calc_start_time = time()
for i in range(0,data_size):
        filename = images_deepsolareye[i]['file_name'].split('/')[2]
        outfile = folder_out + "/" + filename
        outfile_mask = folder_out_mask + "/" + filename
        outfile_rec = folder_out_rec + "/"  + filename
        
        
        image = Image.open(folder + '/' + filename)
        image.save(outfile)
        #print(image.getbbox())
        im_rotate = image.rotate(9)
        box = (20, 20, 180, 160)
        cropped_image = im_rotate.crop(box).resize((192,192))
        cropped_image.save(outfile_rec)
        if i % 1000 == 0:
            print("Number of Images croped", i)
log.log_task_end('calc_deepsolareye_time', calc_start_time)
            

In [9]:
# Make Regression on recification deep solar eye pictures
folder = "data/DeepSolarEyeCalc/DeepSolarEye_192x192_rec"

In [10]:
folder

'data/DeepSolarEyeCalc/DeepSolarEye_192x192_rec'

In [11]:
model_name = 'DeepSolarEyeCalc'
training_start_time = time()
model, mse, rmse, mae = train_keras(model_name, folder, nr_images_batch, nr_batch, nr_epoch, 25)
training_time = time() - training_start_time
log.log_task_end('regression_time', training_start_time)
print("--- %s seconds ---" % (training_time))

Trial 6 Complete [00h 00m 56s]
val_loss: 3.6502622151601827e-06

Best val_loss So Far: 7.162654800871304e-13
Total elapsed time: 00h 09m 38s
Epoch 1/2
200/200 [==============================] - 19s 85ms/step - loss: 0.1551 - mean_squared_error: 0.1551
Epoch 2/2
200/200 [==============================] - 17s 86ms/step - loss: 7.1765e-04 - mean_squared_error: 7.1765e-04
--- Batch 2 --- 
select from to 10000 15000
Files in train_files: 5000
1000 images to array
2000 images to array
3000 images to array
4000 images to array
5000 images to array
All images to array!
Epoch 1/2
200/200 [==============================] - 19s 85ms/step - loss: 0.0117 - mean_squared_error: 0.0117
Epoch 2/2
200/200 [==============================] - 17s 86ms/step - loss: 0.1218 - mean_squared_error: 0.1218
--- Batch 3 --- 
select from to 15000 20000
Files in train_files: 5000
1000 images to array
2000 images to array
3000 images to array
4000 images to array
5000 images to array
All images to array!
Epoch 1/2
200

In [12]:
# Print Model Summary:
print("MODEL SUMMARY")
print("-------------")
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)

MODEL SUMMARY
-------------
Test MSE: 0.0046223784
Test RMSE: 0.067988075
Test MAE: 0.0009615527


In [13]:
# add Metric to the log
log.add_metric('Regression MSE', mse)
log.add_metric('Regression RMSE', rmse)
log.add_metric('Regression MAE', mae)

In [14]:
keras_model = model.export_model()

In [15]:
keras_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 192, 3)]     0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 192, 192, 3)       0         
_________________________________________________________________
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 73729     
Total params: 20,935,209
Trainable params: 20,880,681
Non-trainable params: 54,528
_________________________________________________________________


In [16]:
log.log_experiment_end()

In [17]:
keras_model.save('model/' + str(model_name))